In [3]:
from sklearn.linear_model import RidgeCV
from tqdm import tqdm
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import matplotlib.ticker as ticker
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning

from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
import numpy as np
import pandas as pd

In [4]:
imputed_data = pd.read_csv("sbux_clv_drop_AOV_and_completed_offers_across_channels.csv")
imputed_data=imputed_data.drop(['Unnamed: 0','Customer ID','CLV_Quantile'],axis=1)
imputed_data.head()

,Age,Gender,HH Income,MemberSince,Recency (# Days ago from last trans. As of final day up to Day 15),Frequency (# Trans. from Day 1-15),Monetary Value (Sum of Trans. from Day 1-15),# of Marketing offers that Starbucks sent to each customer from Day 1-15,# of Marketing offers that were viewed from Day 1-15,# of Marketing offers that were successfully completed from Day 1-15,Marketing Offer View Rate from Day 1-15,Marketing Offer Response Rate from Day 1-15,Sum(Trans. Amt from Day 16-30)
0,18-34,M,50k-75k,2017,5,1,22.16,2.0,1.0,0.0,0.50,0.00,105.44
1,Unknown,Unknown,Unknown,2018,2,1,0.70,1.0,1.0,0.0,1.00,0.00,3.39
2,35-50,O,50k-75k,2018,4,2,25.42,3.0,3.0,1.0,1.00,0.33,54.04
3,51-67,F,75k-100k,2016,8,4,98.33,3.0,1.0,1.0,0.33,0.33,98.53
4,18-34,F,50k-75k,2016,2,5,62.86,2.0,2.0,1.0,1.00,0.50,91.19


In [5]:
# Convert categorical variables into dummies
imputed_data = pd.get_dummies(imputed_data)

# Display all the columns in the DataFrame
print(imputed_data.columns)

# Display the modified DataFrame
imputed_data.head()

Index(['MemberSince',
       'Recency (# Days ago from last trans. As of final day up to Day 15)',
       'Frequency (# Trans. from Day 1-15)',
       'Monetary Value (Sum of Trans. from Day 1-15)',
       '# of Marketing offers that Starbucks sent to each customer from Day 1-15',
       '# of Marketing offers that were viewed from Day 1-15',
       '# of Marketing offers that were successfully completed from Day 1-15',
       'Marketing Offer View Rate from Day 1-15',
       'Marketing Offer Response Rate from Day 1-15',
       'Sum(Trans. Amt from Day 16-30)', 'Age_18-34', 'Age_35-50', 'Age_51-67',
       'Age_68-84', 'Age_85-101', 'Age_Unknown', 'Gender_F', 'Gender_M',
       'Gender_O', 'Gender_Unknown', 'HH Income_100k-120k',
       'HH Income_30k-50k', 'HH Income_50k-75k', 'HH Income_75k-100k',
       'HH Income_Unknown'],
      dtype='object')


,MemberSince,Recency (# Days ago from last trans. As of final day up to Day 15),Frequency (# Trans. from Day 1-15),Monetary Value (Sum of Trans. from Day 1-15),# of Marketing offers that Starbucks sent to each customer from Day 1-15,# of Marketing offers that were viewed from Day 1-15,# of Marketing offers that were successfully completed from Day 1-15,Marketing Offer View Rate from Day 1-15,Marketing Offer Response Rate from Day 1-15,Sum(Trans. Amt from Day 16-30),...,Age_Unknown,Gender_F,Gender_M,Gender_O,Gender_Unknown,HH Income_100k-120k,HH Income_30k-50k,HH Income_50k-75k,HH Income_75k-100k,HH Income_Unknown
0,2017,5,1,22.16,2.0,1.0,0.0,0.50,0.00,105.44,...,0,0,1,0,0,0,0,1,0,0
1,2018,2,1,0.70,1.0,1.0,0.0,1.00,0.00,3.39,...,1,0,0,0,1,0,0,0,0,1
2,2018,4,2,25.42,3.0,3.0,1.0,1.00,0.33,54.04,...,0,0,0,1,0,0,0,1,0,0
3,2016,8,4,98.33,3.0,1.0,1.0,0.33,0.33,98.53,...,0,1,0,0,0,0,0,0,1,0
4,2016,2,5,62.86,2.0,2.0,1.0,1.00,0.50,91.19,...,0,1,0,0,0,0,0,1,0,0


In [7]:
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm
import pandas as pd

# Separate the independent variables (features) from the dependent variable (target)
X = imputed_data.drop('Sum(Trans. Amt from Day 16-30)', axis=1)
y = imputed_data['Sum(Trans. Amt from Day 16-30)']

# Create an instance of DecisionTreeRegressor
estimator = DecisionTreeRegressor()

# Create an instance of RFECV and pass in the estimator and number of cross-validation folds
selector = RFECV(estimator, cv=5)

# Fit the selector to the data with a progress bar
for i in tqdm(range(1, X.shape[1] + 1)):
    selector.n_features_ = i
    selector.fit(X, y)

# Get the mask of selected features
feature_mask = selector.support_

# Get the names of the selected features
selected_features = X.columns[feature_mask]

# Print the optimal number of features and their names
print("Optimal number of features: {}".format(selector.n_features_))
print("Selected features:")
for feature in selected_features:
    print(feature)


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [04:57<00:00, 12.39s/it]

Optimal number of features: 1
Selected features:
Monetary Value (Sum of Trans. from Day 1-15)
